In [17]:
# ==============================================================================
# 1. IMPORTAÇÕES E LOGGER PARA PARALELISMO
# ==============================================================================

import os
import time
import logging
import threading
from typing import List
import pandas as pd
import csv
import unicodedata
import re
from concurrent.futures import ThreadPoolExecutor, as_completed

import numpy # Usado para dividir a lista de tarefas
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, NoSuchElementException

from webdriver_manager.chrome import ChromeDriverManager

# Objeto de armazenamento de dados que é local (privado) para cada thread
log_context = threading.local()

class TaskIdFilter(logging.Filter):
    """Filtro para adicionar um ID de tarefa/thread aos registros de log."""
    def filter(self, record: logging.LogRecord) -> bool:
        record.task_id = getattr(log_context, 'task_id', 'MainThread')
        return True

def setup_logging(log_file: str) -> logging.Logger:
    os.makedirs(os.path.dirname(log_file), exist_ok=True)
    logger = logging.getLogger('osr_pacatuba')
    logger.setLevel(logging.INFO)

    if logger.hasHandlers():
        logger.handlers.clear()

    formatter = logging.Formatter("[%(task_id)s] - %(asctime)s - %(levelname)s - [%(funcName)s] - %(message)s")
    task_filter = TaskIdFilter()

    file_handler = logging.FileHandler(log_file, mode='w', encoding='utf-8')
    file_handler.setFormatter(formatter)
    file_handler.addFilter(task_filter)

    console_handler = logging.StreamHandler()
    console_handler.setFormatter(formatter)
    console_handler.addFilter(task_filter)

    logger.addHandler(file_handler)
    logger.addHandler(console_handler)
    
    # Reduz o ruído de bibliotecas externas
    logging.getLogger('selenium').setLevel(logging.WARNING)
    logging.getLogger('urllib3').setLevel(logging.WARNING)
    logging.getLogger('WDM').setLevel(logging.WARNING)

    return logger

# A variável logger será inicializada dentro da função de orquestração
logger = None

In [18]:
# ==============================================================================
# 2. CONSTANTES E FUNÇÕES AUXILIARES
# ==============================================================================

# Agora os termos são os códigos, conforme seu código
TERMOS_ROYALTIES = ["royaltie", "royalty", "petroleo"]

RE_REMOVE_PUNCTUATION = re.compile(r'[^a-zA-Z0-9\s]')

def normalizar(texto: str) -> str:
    """
    Normaliza um texto removendo acentos, pontuações e convertendo para minúsculas.
    """
    if not isinstance(texto, str):
        return ""
    texto = unicodedata.normalize('NFKD', texto).encode('ASCII', 'ignore').decode('utf-8')
    texto = RE_REMOVE_PUNCTUATION.sub('', texto)
    return texto.lower()

In [19]:
# ==============================================================================
# 3. DRIVER E INTERAÇÕES COM A PÁGINA (start_driver ATUALIZADO)
# ==============================================================================

def start_driver(headless=False) -> webdriver.Chrome:
    logger.info("Iniciando e configurando instância do Chrome...")
    options = webdriver.ChromeOptions()

    if headless:
        options.add_argument("--headless=new")
        options.add_argument("--disable-gpu")
        options.add_argument("--no-sandbox")
    else:
        options.add_argument('--window-size=1920,1080')
        options.add_argument("--disable-blink-features=AutomationControlled")
        options.add_experimental_option("excludeSwitches", ["enable-automation"])
    
    # A função agora apenas cria e retorna o driver, sem navegar.
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
    return driver


def selecionar_dropdown(driver, container_id, texto):
    try:
        logger.info(f"Selecionando: {texto}")
        trigger = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, f"//span[@aria-labelledby='{container_id}']"))
        )
        trigger.click()
        opcao = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, f"//li[contains(@class, 'select2-results__option') and normalize-space(.)='{texto}']"))
        )
        opcao.click()
    except Exception as e:
        logger.error(f"Erro ao selecionar {texto} no campo {container_id}: {e}")
        raise

def ir_para_proxima_pagina(driver):
    try:
        proxima_pagina_locator = (By.XPATH, "//a[contains(@class, 'page-link')][i[contains(@class, 'next')]]")
        botao = driver.find_element(*proxima_pagina_locator)
        parent_li = botao.find_element(By.XPATH, "./parent::li")
        
        if "disabled" in parent_li.get_attribute("class"):
            logger.info("Última página alcançada.")
            return False

        botao.click()
        logger.info("Navegando para a próxima página de resultados.")
        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//table/tbody")))
        return True
    except NoSuchElementException:
        logger.info("Botão 'Próxima Página' não encontrado. Fim da paginação.")
        return False

In [ ]:
# ==============================================================================
# 4. WORKER PARALELO E ORQUESTRAÇÃO
# ==============================================================================
def worker_extrair_detalhes(links: List[str], ano_alvo: str) -> List[dict]:
    """
    Função 'worker' para threads. Otimizada para verificar a fonte de recurso
    primeiro, antes de extrair todos os outros dados.
    """
    log_context.task_id = f"Worker-{threading.get_ident() % 1000}"
    logger = logging.getLogger('osr_pacatuba')
    
    logger.info(f"Worker iniciado. Processando {len(links)} links.")
    
    dados_coletados_pela_thread = []
    driver = None
    try:
        driver = start_driver(headless=True)
        
        for i, link in enumerate(links):
            try:
                logger.debug(f"Acessando link {i+1}/{len(links)}.")
                driver.get(link)
                
                # Aguarda o elemento mais básico da página de detalhes para garantir o carregamento
                WebDriverWait(driver, 20).until(
                    EC.visibility_of_element_located((By.ID, "table-dados"))
                )

                # Mapa de XPaths para todos os campos na página de detalhes.
                XPATHS_DETALHES = {           
                    'empenho':          '//*[@id="table-dados"]/tbody/tr[2]/td[1]',
                    'credor':           '//*[@id="table-dados"]/tbody/tr[2]/td[2]',
                    'data_nota':        '//*[@id="table-dados"]/tbody/tr[2]/td[3]',

                    'processo':         '//*[@id="table-dados"]/tbody/tr[4]/th[1]',
                    'fonte_recurso':    '//*[@id="table-dados"]/tbody/tr[4]/th[2]',            
                    'numero_documento': '//*[@id="table-dados"]/tbody/tr[4]/th[3]',

                    'valor_pago':       '//*[@id="table-dados"]/tbody/tr[6]/td[1]',
                    'valor_retido':     '//*[@id="table-dados"]/tbody/tr[6]/td[2]',
                    'forma_pagamento':  '//*[@id="table-dados"]/tbody/tr[6]/td[3]',
                    
                    'historico':        '//*[@id="table-historico"]/tbody/tr/td',
                    'relacionado_covid':'//*[@id="table-outras-informacoes"]/tbody/tr/td[1]',
                    'relacionado_LC173':'//*[@id="table-outras-informacoes"]/tbody/tr/td[2]'
                }
                
                # --- ETAPA 1: Extrair APENAS a Fonte de Recurso para verificação ---
                logger.debug("Verificando a Fonte de Recurso primeiro...")
                fonte_recurso_texto = None
                try:
                    # Usa o XPath específico para a fonte de recurso
                    fonte_recurso_element = driver.find_element(By.XPATH, XPATHS_DETALHES['fonte_recurso'])
                    fonte_recurso_texto = fonte_recurso_element.text.strip()
                    fonte_recurso_texto = normalizar(fonte_recurso_texto)
                    
                except NoSuchElementException:
                    logger.warning(f"Campo 'fonte_recurso' não encontrado no link {link}. Pulando.")
                    continue # Pula para o próximo link

                # --- ETAPA 2: Verificar se é de royalties ANTES de extrair o resto ---
                if fonte_recurso_texto and any(termo in fonte_recurso_texto for termo in TERMOS_ROYALTIES):
                    logger.info(f"Royalties encontrados (Fonte: '{fonte_recurso_texto}'). Extraindo todos os dados do link: {link}")
                    
                    # --- ETAPA 3: Se for royalties, extrair todos os outros campos ---
                    dados_completos = {'fonte_recurso': fonte_recurso_texto}
                    
                    for nome_campo, xpath in XPATHS_DETALHES.items():
                        if nome_campo == 'fonte_recurso': # Já pegamos este
                            continue
                        try:
                            dados_completos[nome_campo] = driver.find_element(By.XPATH, xpath).text.strip()
                        except NoSuchElementException:
                            dados_completos[nome_campo] = None
                    
                    dados_completos['link_detalhe'] = link
                    dados_coletados_pela_thread.append(dados_completos)

                else:
                    logger.debug(f"Link não é de royalties. Fonte: '{fonte_recurso_texto}'. Pulando extração detalhada.")

            except Exception as e_link:
                logger.error(f"Erro ao processar o link {link}: {e_link}")
                continue # Continua para o próximo link
    finally:
        if driver: 
            driver.quit()
        logger.info("Worker finalizado.")
    
    return dados_coletados_pela_thread


def processar_ano_em_paralelo(ano_alvo: str, max_workers: int = 4):
    """
    Orquestra o processo: coleta todos os links e depois distribui a extração.
    """
    global logger
    logger = setup_logging(log_file=f"../logs/pacatuba/extracao_{ano_alvo}.log")
    
    logger.info(f"--- INICIANDO PROCESSAMENTO PARALELO PARA O ANO DE {ano_alvo} ---")
    
    # --- FASE 1: COLETA DE TODOS OS LINKS NA THREAD PRINCIPAL ---
    links_para_processar = []
    driver = None
    try:
        logger.info("Fase 1: Coletando todos os links de detalhes...")
        driver = start_driver(headless=True)
        url_incial = "https://transparencia.pacatuba.se.gov.br/public/portal/despesas"
        driver.get(url_incial)
        WebDriverWait(driver, 15).until(EC.visibility_of_element_located((By.TAG_NAME, 'body')))
        
        selecionar_dropdown(driver, "select2-tipo-container", "Pagamento")
        selecionar_dropdown(driver, "select2-ano-container", ano_alvo)
        
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button#filtrar.btn-buscar"))).click()
        WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//table/tbody")))
        
        pagina_atual = 1
        while True:
            logger.info(f"Coletando links da página de resultados: {pagina_atual}")
            
            # =================================================================
            # AQUI ESTÁ A CORREÇÃO
            # =================================================================
            # Usando o seletor XPath específico para a coluna de detalhes
            botoes_detalhes_locator = (By.XPATH, "//td[@serigyitem='detalhesPagamento']/a")
            botoes_detalhes = WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located(botoes_detalhes_locator)
            )
            # =================================================================
            
            for botao in botoes_detalhes:
                if link := botao.get_attribute('href'): 
                    links_para_processar.append(link)
            #break
            if not ir_para_proxima_pagina(driver): 
                break
            pagina_atual += 1
            
    finally:
        if driver: 
            driver.quit()
    
    logger.info(f"Fase 1 concluída. Total de {len(links_para_processar)} links de detalhes coletados.")
    
    if not links_para_processar: 
        logger.warning("Nenhum link para processar. Encerrando.")
        return

    # --- FASE 2: DISTRIBUIÇÃO E PROCESSAMENTO PARALELO ---
    # (Esta parte do código permanece a mesma, pois o erro estava na coleta dos links)
    logger.info(f"Fase 2: Iniciando extração com {max_workers} workers.")
    dados_finais = []
    
    if max_workers > len(links_para_processar): max_workers = len(links_para_processar)
    
    lista_de_tarefas = numpy.array_split(links_para_processar, max_workers) if max_workers > 0 else [links_para_processar]
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(worker_extrair_detalhes, tarefa.tolist(), ano_alvo): i for i, tarefa in enumerate(lista_de_tarefas) if tarefa.size > 0}
        
        for future in tqdm(as_completed(futures), total=len(futures), desc=f"Processando Detalhes de {ano_alvo}"):
            if resultado_parcial := future.result(): 
                dados_finais.extend(resultado_parcial)

    # --- SALVAR RESULTADOS ---
    if dados_finais:
        output_dir = os.path.join("dados_finais", "pacatuba")
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, f"royalties_pacatuba_{ano_alvo}.csv")
        
        df = pd.DataFrame(dados_finais)
        df.to_csv(output_path, index=False, sep=';', encoding='utf-8-sig')
        logger.info(f"Processamento concluído. {len(df)} registros salvos em: {output_path}")
    else:
        logger.info("Nenhum registro de royalties foi extraído.")
        
    logger.info(f"--- FINALIZADO PROCESSAMENTO DE {ano_alvo} ---")

In [16]:
# ==============================================================================
# 5. EXECUÇÃO DO SCRIPT
# ==============================================================================

if __name__ == "__main__":
    ANO_A_PROCESSAR = "2018"
    NUMERO_DE_THREADS = 2 # Defina quantos navegadores rodarão em paralelo

    processar_ano_em_paralelo(
        ano_alvo=ANO_A_PROCESSAR,
        max_workers=NUMERO_DE_THREADS
    )

[MainThread] - 2025-07-14 12:08:35,924 - INFO - [processar_ano_em_paralelo] - --- INICIANDO PROCESSAMENTO PARALELO PARA O ANO DE 2018 ---
[MainThread] - 2025-07-14 12:08:35,926 - INFO - [processar_ano_em_paralelo] - Fase 1: Coletando todos os links de detalhes...
[MainThread] - 2025-07-14 12:08:35,927 - INFO - [start_driver] - Iniciando e configurando instância do Chrome...
[MainThread] - 2025-07-14 12:08:52,043 - INFO - [selecionar_dropdown] - Selecionando: Pagamento
[MainThread] - 2025-07-14 12:08:52,264 - INFO - [selecionar_dropdown] - Selecionando: 2018
[MainThread] - 2025-07-14 12:09:03,924 - INFO - [processar_ano_em_paralelo] - Coletando links da página de resultados: 1
[MainThread] - 2025-07-14 12:09:15,587 - INFO - [ir_para_proxima_pagina] - Navegando para a próxima página de resultados.
[MainThread] - 2025-07-14 12:09:15,610 - INFO - [processar_ano_em_paralelo] - Coletando links da página de resultados: 2
[MainThread] - 2025-07-14 12:09:28,046 - INFO - [ir_para_proxima_pagina]

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x0x9e44a3+62419]
	GetHandleVerifier [0x0x9e44e4+62484]
	(No symbol) [0x0x822133]
	(No symbol) [0x0x86a8fe]
	(No symbol) [0x0x86ac9b]
	(No symbol) [0x0x8b3052]
	(No symbol) [0x0x88f4b4]
	(No symbol) [0x0x8b087a]
	(No symbol) [0x0x88f266]
	(No symbol) [0x0x85e852]
	(No symbol) [0x0x85f6f4]
	GetHandleVerifier [0x0xc54793+2619075]
	GetHandleVerifier [0x0xc4fbaa+2599642]
	GetHandleVerifier [0x0xa0b04a+221050]
	GetHandleVerifier [0x0x9fb2c8+156152]
	GetHandleVerifier [0x0xa01c7d+183213]
	GetHandleVerifier [0x0x9ec388+94904]
	GetHandleVerifier [0x0x9ec512+95298]
	GetHandleVerifier [0x0x9d766a+9626]
	BaseThreadInitThunk [0x0x762b7ba9+25]
	RtlInitializeExceptionChain [0x0x7761c28b+107]
	RtlClearBits [0x0x7761c20f+191]
